In [34]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import keras
import numpy as np
import pandas as pd
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, BatchNormalization
from keras.layers.core import Permute, Reshape
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import Regularizer
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.utils import to_categorical

%matplotlib inline  

### Opening Face Points

In [35]:
FILENAME = 'features_points.h5'
h5f = h5py.File(FILENAME,'r')

for key in h5f.keys():
    print(key) #Names of the groups in HDF5 file.
x = h5f['points'][:]

points


In [36]:
x.shape

(614, 25, 68, 2)

### Getting targets in System

In [37]:
targets = pd.read_csv('merged_labels.csv')
targets.head()


,SessionID,Session,Type,Merged Arousal,Merged Dominance,Merged Emotion
0,S001-001-l000,1,Laughter,1,1,Happy
1,S001-001-l001,1,Laughter,1,1,Happy
2,S001-001-l002,1,Laughter,2,2,Happy
3,S001-003-l000,3,Laughter,1,1,Schadenfreude
4,S001-003-l001,3,Laughter,1,2,Schadenfreude


In [38]:
y = targets['Merged Arousal']
y1 = targets['Merged Dominance']
y2 = targets['Merged Emotion']

In [39]:
def create_graph_arousal_dominance():
    
    model.add(BatchNormalization(input_shape=(feat_dim,WINDOW_SIZE,1)))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 5),
                 activation='relu',
                 padding='same'))
    #model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(num_feat_map, kernel_size=(1, 5), activation='relu',padding='same'))
    #model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    
    model.add(Permute((2, 1, 3))) # for swap-dimension
    model.add(Reshape((-1,num_feat_map*feat_dim)))

    model.add(LSTM(num_feat_map, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='linear'))
    
    model.summary()
    

# Simple LSTM Model Arousal

In [40]:
WINDOW_SIZE = 25
num_feat_map = 32
feat_dim = 136

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = x_train.reshape((491,25,-1))
x_test = x_test.reshape((123,25,-1))

x_train = np.swapaxes(x_train,1,2)
print(x_train.shape)
x_train = np.reshape(x_train, (-1, feat_dim, WINDOW_SIZE, 1))
x_test = np.swapaxes(x_test,1,2)
x_test = np.reshape(x_test, (-1, feat_dim, WINDOW_SIZE, 1))

model = Sequential()

create_graph_arousal_dominance()

(491, 136, 25)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_12 (Batc (None, 136, 25, 1)        4         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 136, 25, 32)       192       
_________________________________________________________________
dropout_19 (Dropout)         (None, 136, 25, 32)       0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 136, 25, 32)       128       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 136, 25, 32)       5152      
_________________________________________________________________
dropout_20 (Dropout)         (None, 136, 25, 32)       0         
_________________________________________________________________
permute_7 (Permute)          (None, 25, 136, 32)       0     

In [42]:
epochs = 150
batch_size=16

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
#filepath="weights_best_arousal.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.compile(loss=keras.losses.mean_squared_error,
              optimizer='adam',
              metrics=['mse'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=(x_test, y_test))

Train on 491 samples, validate on 123 samples
Epoch 1/150
491/491 [==============================] - 6s - loss: 1.9216 - mean_squared_error: 1.9216 - val_loss: 1.3890 - val_mean_squared_error: 1.3890
Epoch 2/150
491/491 [==============================] - 2s - loss: 1.7723 - mean_squared_error: 1.7723 - val_loss: 1.3887 - val_mean_squared_error: 1.3887
Epoch 3/150
491/491 [==============================] - 2s - loss: 1.6453 - mean_squared_error: 1.6453 - val_loss: 1.4549 - val_mean_squared_error: 1.4549
Epoch 4/150
491/491 [==============================] - 2s - loss: 1.6392 - mean_squared_error: 1.6392 - val_loss: 1.4312 - val_mean_squared_error: 1.4312
Epoch 5/150
491/491 [==============================] - 2s - loss: 1.5700 - mean_squared_error: 1.5700 - val_loss: 1.2968 - val_mean_squared_error: 1.2968
Epoch 6/150
491/491 [==============================] - 2s - loss: 1.5010 - mean_squared_error: 1.5010 - val_loss: 1.3077 - val_mean_squared_error: 1.3077
Epoch 7/150
491/491 [=========

491/491 [==============================] - 2s - loss: 0.9992 - mean_squared_error: 0.9992 - val_loss: 0.8974 - val_mean_squared_error: 0.8974
Epoch 54/150
491/491 [==============================] - 2s - loss: 1.0272 - mean_squared_error: 1.0272 - val_loss: 1.0038 - val_mean_squared_error: 1.0038
Epoch 55/150
491/491 [==============================] - 2s - loss: 0.9860 - mean_squared_error: 0.9860 - val_loss: 0.7840 - val_mean_squared_error: 0.7840
Epoch 56/150
491/491 [==============================] - 2s - loss: 1.0123 - mean_squared_error: 1.0123 - val_loss: 0.7662 - val_mean_squared_error: 0.7662
Epoch 57/150
491/491 [==============================] - 2s - loss: 0.9811 - mean_squared_error: 0.9811 - val_loss: 0.7721 - val_mean_squared_error: 0.7721
Epoch 58/150
491/491 [==============================] - 2s - loss: 1.0562 - mean_squared_error: 1.0562 - val_loss: 1.0797 - val_mean_squared_error: 1.0797
Epoch 59/150
491/491 [==============================] - 2s - loss: 0.9956 - mean_sq

491/491 [==============================] - 2s - loss: 0.8966 - mean_squared_error: 0.8966 - val_loss: 0.7645 - val_mean_squared_error: 0.7645
Epoch 106/150
491/491 [==============================] - 2s - loss: 0.8687 - mean_squared_error: 0.8687 - val_loss: 0.6828 - val_mean_squared_error: 0.6828
Epoch 107/150
491/491 [==============================] - 2s - loss: 0.7872 - mean_squared_error: 0.7872 - val_loss: 0.6790 - val_mean_squared_error: 0.6790
Epoch 108/150
491/491 [==============================] - 2s - loss: 0.8484 - mean_squared_error: 0.8484 - val_loss: 0.7413 - val_mean_squared_error: 0.7413
Epoch 109/150
491/491 [==============================] - 2s - loss: 0.9339 - mean_squared_error: 0.9339 - val_loss: 0.8484 - val_mean_squared_error: 0.8484
Epoch 110/150
491/491 [==============================] - 2s - loss: 0.8445 - mean_squared_error: 0.8445 - val_loss: 0.7444 - val_mean_squared_error: 0.7444
Epoch 111/150
491/491 [==============================] - 2s - loss: 0.8495 - m

In [43]:
preds = model.predict(x_test)
preds = preds.reshape(preds.shape[0])

#### Prediction Eval

In [44]:
mean_y = np.mean(preds)
print("Mean of predictions is: "+str(mean_y))

mse = np.mean((y_test - preds)**2)
print("Variance (benchmark) is: "+str(mse))

Mean of predictions is: 1.62584
Variance (benchmark) is: 0.8619084812746621


In [45]:
preds

array([ 2.95983934,  2.33952212,  1.49513865,  2.8971839 ,  1.1220454 ,
        1.38333464,  1.03301454,  1.53231645,  1.00133991,  1.16833699,
        1.06483388,  1.17436779,  1.22689211,  1.66218305,  1.03690219,
        3.27933645,  1.91843796,  1.56104755,  1.01654756,  1.03221893,
        1.11031806,  2.91095638,  1.34503186,  3.16199493,  2.78793812,
        1.33625317,  1.09390211,  1.22254777,  1.60882664,  1.60620427,
        1.0125407 ,  1.00012457,  2.09408283,  2.17011309,  1.09889877,
        1.56255245,  1.93120563,  1.27455807,  1.66139865,  1.05828881,
        1.38415611,  1.17935944,  1.15163648,  1.00388551,  1.19712543,
        1.48689771,  3.1446228 ,  1.43105936,  1.52753067,  3.05424404,
        1.03209972,  2.16394949,  1.32477295,  3.0854764 ,  1.02196085,
        0.99846882,  1.11588609,  1.36303031,  1.70385456,  1.68173981,
        0.99650627,  2.96879864,  1.04304695,  0.98537302,  2.5808754 ,
        2.61990428,  2.69581366,  1.26664519,  1.78058434,  1.00

#### Benchmark Eval

In [46]:
mean_y = np.mean(y_test)
print("Mean of test set is: "+str(mean_y))

variance = np.mean((y_test - np.mean(y_test))**2)
print("Variance (benchmark) is: "+str(variance))

Mean of test set is: 1.886178861788618
Variance (benchmark) is: 1.3691585696344772


# Simple LSTM Dominance

In [47]:
WINDOW_SIZE = 25
num_feat_map = 32
feat_dim = 136

model = Sequential()

x_train, x_test, y_train, y_test = train_test_split(x, y1, test_size=0.2, random_state=42)

x_train = x_train.reshape((491,25,-1))
x_test = x_test.reshape((123,25,-1))

x_train = np.swapaxes(x_train,1,2)
x_train = np.reshape(x_train, (-1, feat_dim, WINDOW_SIZE, 1))
x_test = np.swapaxes(x_test,1,2)
x_test = np.reshape(x_test, (-1, feat_dim, WINDOW_SIZE, 1))

create_graph_arousal_dominance()

epochs = 200
batch_size=16

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
sgd = keras.optimizers.SGD(lr=0.01, clipnorm=1.)
filepath="weights_best_arousal.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#adam1 = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss=keras.losses.mean_squared_error,
              optimizer='adam',
              metrics=['mse'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_14 (Batc (None, 136, 25, 1)        4         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 136, 25, 32)       192       
_________________________________________________________________
dropout_22 (Dropout)         (None, 136, 25, 32)       0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 136, 25, 32)       128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 136, 25, 32)       5152      
_________________________________________________________________
dropout_23 (Dropout)         (None, 136, 25, 32)       0         
_________________________________________________________________
permute_8 (Permute)          (None, 25, 136, 32)       0         
__________

491/491 [==============================] - 2s - loss: 0.8000 - mean_squared_error: 0.8000 - val_loss: 0.6485 - val_mean_squared_error: 0.6485
Epoch 43/200
491/491 [==============================] - 2s - loss: 0.7631 - mean_squared_error: 0.7631 - val_loss: 0.6277 - val_mean_squared_error: 0.6277
Epoch 44/200
491/491 [==============================] - 2s - loss: 0.8276 - mean_squared_error: 0.8276 - val_loss: 0.6296 - val_mean_squared_error: 0.6296
Epoch 45/200
491/491 [==============================] - 2s - loss: 0.7887 - mean_squared_error: 0.7887 - val_loss: 0.6077 - val_mean_squared_error: 0.6077
Epoch 46/200
491/491 [==============================] - 2s - loss: 0.8264 - mean_squared_error: 0.8264 - val_loss: 0.6836 - val_mean_squared_error: 0.6836
Epoch 47/200
491/491 [==============================] - 2s - loss: 0.8532 - mean_squared_error: 0.8532 - val_loss: 0.6453 - val_mean_squared_error: 0.6453
Epoch 48/200
491/491 [==============================] - 2s - loss: 0.7771 - mean_sq

491/491 [==============================] - 2s - loss: 0.6880 - mean_squared_error: 0.6880 - val_loss: 0.6136 - val_mean_squared_error: 0.6136
Epoch 95/200
491/491 [==============================] - 2s - loss: 0.7095 - mean_squared_error: 0.7095 - val_loss: 0.6438 - val_mean_squared_error: 0.6438
Epoch 96/200
491/491 [==============================] - 2s - loss: 0.7250 - mean_squared_error: 0.7250 - val_loss: 0.6243 - val_mean_squared_error: 0.6243
Epoch 97/200
491/491 [==============================] - 2s - loss: 0.7051 - mean_squared_error: 0.7051 - val_loss: 0.6689 - val_mean_squared_error: 0.6689
Epoch 98/200
491/491 [==============================] - 2s - loss: 0.7117 - mean_squared_error: 0.7117 - val_loss: 0.6155 - val_mean_squared_error: 0.6155
Epoch 99/200
491/491 [==============================] - 2s - loss: 0.7061 - mean_squared_error: 0.7061 - val_loss: 0.6170 - val_mean_squared_error: 0.6170
Epoch 100/200
491/491 [==============================] - 2s - loss: 0.6617 - mean_s

491/491 [==============================] - 2s - loss: 0.5873 - mean_squared_error: 0.5873 - val_loss: 0.6855 - val_mean_squared_error: 0.6855
Epoch 147/200
491/491 [==============================] - 2s - loss: 0.6234 - mean_squared_error: 0.6234 - val_loss: 0.6638 - val_mean_squared_error: 0.6638
Epoch 148/200
491/491 [==============================] - 2s - loss: 0.5989 - mean_squared_error: 0.5989 - val_loss: 0.7085 - val_mean_squared_error: 0.7085
Epoch 149/200
491/491 [==============================] - 2s - loss: 0.6348 - mean_squared_error: 0.6348 - val_loss: 0.7138 - val_mean_squared_error: 0.7138
Epoch 150/200
491/491 [==============================] - 2s - loss: 0.6278 - mean_squared_error: 0.6278 - val_loss: 0.7110 - val_mean_squared_error: 0.7110
Epoch 151/200
491/491 [==============================] - 2s - loss: 0.6167 - mean_squared_error: 0.6167 - val_loss: 0.6941 - val_mean_squared_error: 0.6941
Epoch 152/200
491/491 [==============================] - 2s - loss: 0.6080 - m

491/491 [==============================] - 2s - loss: 0.5428 - mean_squared_error: 0.5428 - val_loss: 0.7030 - val_mean_squared_error: 0.7030
Epoch 199/200
491/491 [==============================] - 2s - loss: 0.5510 - mean_squared_error: 0.5510 - val_loss: 0.7133 - val_mean_squared_error: 0.7133
Epoch 200/200
491/491 [==============================] - 2s - loss: 0.5801 - mean_squared_error: 0.5801 - val_loss: 0.7119 - val_mean_squared_error: 0.7119


#### Eval

In [48]:
preds = model.predict(x_test)
preds = preds.reshape(preds.shape[0])

mean_y = np.mean(preds)
print("Mean of predictions is: "+str(mean_y))

mse = np.mean((y_test - preds)**2)
print("Variance (benchmark) is: "+str(mse))

print()

mean_y = np.mean(y_test)
print("Mean of test set is: "+str(mean_y))

variance = np.mean((y_test - np.mean(y_test))**2)
print("Variance (benchmark) is: "+str(variance))

Mean of predictions is: 2.14164
Variance (benchmark) is: 0.7118983274968411

Mean of test set is: 2.3983739837398375
Variance (benchmark) is: 0.7437371934694955


In [49]:
preds

array([ 2.22657466,  2.58705044,  2.3368082 ,  1.65678477,  1.68652165,
        1.92236519,  2.69787979,  2.28533244,  1.99574423,  2.03144789,
        2.41894126,  1.83183014,  2.09357214,  2.76765251,  1.58795977,
        0.95379198,  2.5035367 ,  2.50261521,  1.99242985,  1.99649405,
        2.17121124,  2.09543419,  1.74592948,  3.0868721 ,  2.55323148,
        2.45642519,  1.63740265,  2.42527056,  2.03338981,  1.90654135,
        1.86166656,  1.88584161,  2.85888004,  2.09000206,  1.97785497,
        2.29065275,  2.55164099,  1.70625722,  1.99755561,  1.97617173,
        2.15045929,  2.08101177,  2.72565508,  2.00720668,  2.03776836,
        2.33207512,  2.69371414,  2.16109204,  2.51240921,  1.92054772,
        2.01066041,  2.58390236,  1.93260086,  2.43810773,  1.98575175,
        1.8719089 ,  2.11988235,  2.47366667,  1.81415975,  2.47585821,
        1.99028111,  2.00997686,  2.06720328,  1.96771026,  2.66484714,
        2.34639311,  2.28281927,  1.89723563,  1.58547139,  2.05

# Simple LSTM Model Emotions

In [50]:
def create_graph_emotions():
    
    model.add(BatchNormalization(input_shape=(feat_dim,WINDOW_SIZE,1)))
    
    model.add(Conv2D(num_feat_map, kernel_size=(1, 5),
                 activation='relu',
                 padding='same'))
    #model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Conv2D(num_feat_map, kernel_size=(1, 5), activation='relu',padding='same'))
    #model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    
    model.add(Permute((2, 1, 3))) # for swap-dimension
    model.add(Reshape((-1,num_feat_map*feat_dim)))

    model.add(LSTM(num_feat_map, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(4,activation='softmax'))
    
    model.summary()

In [51]:
WINDOW_SIZE = 25
num_feat_map = 32
feat_dim = 136

model = Sequential()

label_encoder = LabelEncoder()
y2 = label_encoder.fit_transform(y2)

x_train, x_test, y_train, y_test = train_test_split(x, y2, test_size=0.2, random_state=42)

x_train = x_train.reshape((491,25,-1))
x_test = x_test.reshape((123,25,-1))

x_train = np.swapaxes(x_train,1,2)
x_train = np.reshape(x_train, (-1, feat_dim, WINDOW_SIZE, 1))
x_test = np.swapaxes(x_test,1,2)
x_test = np.reshape(x_test, (-1, feat_dim, WINDOW_SIZE, 1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

create_graph_emotions()

epochs = 100
batch_size=32

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
#filepath="weights_best_arousal.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


model.compile(loss=keras.losses.mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_16 (Batc (None, 136, 25, 1)        4         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 136, 25, 32)       192       
_________________________________________________________________
dropout_25 (Dropout)         (None, 136, 25, 32)       0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 136, 25, 32)       128       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 136, 25, 32)       5152      
_________________________________________________________________
dropout_26 (Dropout)         (None, 136, 25, 32)       0         
_________________________________________________________________
permute_9 (Permute)          (None, 25, 136, 32)       0         
__________

491/491 [==============================] - 2s - loss: 0.1658 - acc: 0.4562 - val_loss: 0.1592 - val_acc: 0.4959
Epoch 52/100
491/491 [==============================] - 2s - loss: 0.1685 - acc: 0.4664 - val_loss: 0.1605 - val_acc: 0.5203
Epoch 53/100
491/491 [==============================] - 2s - loss: 0.1700 - acc: 0.4623 - val_loss: 0.1576 - val_acc: 0.5203
Epoch 54/100
491/491 [==============================] - 2s - loss: 0.1671 - acc: 0.4705 - val_loss: 0.1611 - val_acc: 0.4959
Epoch 55/100
491/491 [==============================] - 2s - loss: 0.1678 - acc: 0.4684 - val_loss: 0.1589 - val_acc: 0.4959
Epoch 56/100
491/491 [==============================] - 2s - loss: 0.1660 - acc: 0.4562 - val_loss: 0.1597 - val_acc: 0.4959
Epoch 57/100
491/491 [==============================] - 2s - loss: 0.1701 - acc: 0.4562 - val_loss: 0.1602 - val_acc: 0.5041
Epoch 58/100
491/491 [==============================] - 2s - loss: 0.1683 - acc: 0.4705 - val_loss: 0.1609 - val_acc: 0.5203
Epoch 59/100


#### Eval

In [52]:
preds = model.predict_classes(x_test)
preds

 96/123 [======================>.......] - ETA: 0s

array([2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 3, 2, 2, 2, 2,
       0, 2, 3, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 0,
       0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2,
       2, 0, 2, 2, 2, 3, 2, 2], dtype=int64)

In [53]:
y_test = np.argmax(y_test,axis=1)
y_test

array([1, 3, 2, 2, 1, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 3, 2, 2, 0, 3, 2, 0, 0,
       3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 0, 2, 1, 0, 3, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 3, 1, 0, 2, 3, 3, 1, 2, 3, 2,
       0, 1, 3, 0, 2, 2, 2, 2, 0, 2, 0, 3, 0, 0, 2, 2, 3, 2, 2, 3, 1, 0, 3,
       0, 0, 0, 1, 1, 0, 2, 0, 3, 2, 2, 0, 2, 2, 3, 0, 2, 0, 2, 1, 2, 2, 1,
       2, 1, 2, 2, 3, 2, 2, 2], dtype=int64)

In [54]:
confusion_matrix(y_test,preds)

array([[17,  0, 13,  0],
       [ 6,  0,  9,  0],
       [ 8,  0, 52,  1],
       [ 3,  0, 11,  3]], dtype=int64)

In [55]:
print(classification_report(y_test,preds))

             precision    recall  f1-score   support

          0       0.50      0.57      0.53        30
          1       0.00      0.00      0.00        15
          2       0.61      0.85      0.71        61
          3       0.75      0.18      0.29        17

avg / total       0.53      0.59      0.52       123



C:\Users\Sam\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Maybe talk about resampling to take care of class imbalances in the optimization section...